In [ ]:
!pip install -q langchain langchain-groq langchain-community sentence-transformers faiss-cpu pypdf python-dotenv ipywidgets

In [5]:
import os
from getpass import getpass
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from IPython.display import display
import ipywidgets as widgets

In [6]:
os.environ["GROQ_API_KEY"] = "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"

In [7]:
from google.colab import files
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]


ModuleNotFoundError: No module named 'google.colab'

In [ ]:
loader = PyPDFLoader(pdf_path)
docs = loader.load()

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_documents(docs)

In [ ]:
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embedding=embedding)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

/tmp/ipython-input-13-4151374717.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
llm = ChatGroq(
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model_name="llama3-70b-8192",
    temperature=0.5
)

In [ ]:
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
Answer the customer's question using only the following content.

[Context]
{context}

[Question]
{question}

[Answer]
"""
)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:
print("🤖 Chatbot ready! Ask questions about your PDF.")

while True:
    question = input("You: ")
    if question.lower() in ["exit", "quit"]:
        print("👋 Exiting chatbot.")
        break
    response = qa_chain.run(question)
    print("Bot:", response)

🤖 Chatbot ready! Ask questions about your PDF.
You: what is machine learning


/tmp/ipython-input-17-1629861551.py:8: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(question)


Bot: Machine learning (ML) is a branch of artificial intelligence that enables computers to learn from data—without being explicitly programmed. Instead of following rigid instructions, ML systems analyze patterns and improve their performance with experience.
You: their types?
Bot: The types of Machine Learning are:

• Supervised Learning: The model learns from labeled data (e.g., emails marked “spam” or “not spam”).
• Unsupervised Learning: Finds hidden patterns in unlabeled data (e.g., grouping customers by behavior).
• Reinforcement Learning: The model learns by trial and error, receiving feedback—like training a robot to walk.
You: how it works?
Bot: The process of Machine Learning typically involves: 

1. Data Collection: Huge volumes of relevant data are gathered.
2. Training a Model: This data is used to teach the algorithm what patterns to recognize.
3. Evaluation: The model is tested with new data to check its accuracy.
4. Improvement: Based on the results, the model is refin